In [1]:
%Addjar --magic http://9.30.146.148/elasticsearch-spark-20_2.11-5.3.0.jar -f

Starting download from http://9.30.146.148/elasticsearch-spark-20_2.11-5.3.0.jar
Finished download of elasticsearch-spark-20_2.11-5.3.0.jar


In [ ]:
import org.elasticsearch.spark._

In [35]:
import org.apache.spark.SparkConf
val conf =new SparkConf().setAppName("Recommand").setMaster("spark://xmo1.fyre.ibm.com:7077")
    conf.set("es.index.auto.create","true")
    conf.set("es.nodes","9.30.101.216")

In [42]:
ratings_df.show(5)

+-------+------+--------------------+------+
|movieId|rating|           timestamp|userId|
+-------+------+--------------------+------+
|    253|   3.0|1970-01-10 11:23:...|    50|
|    282|   3.0|1970-01-10 11:23:...|    50|
|    368|   4.0|1970-01-10 11:23:...|    50|
|    480|   4.0|1970-01-10 11:23:...|    50|
|    587|   3.0|1970-01-10 11:23:...|    50|
+-------+------+--------------------+------+
only showing top 5 rows



In [45]:
ratings_df.write.format("es").save("demo0/ratings")

In [37]:
val numbers = Map("one" -> 1, "two" -> 2, "three" -> 3)
val airports = Map("arrival" -> "Otopeni", "SFO" -> "San Fran")
sc.makeRDD(Seq(numbers, airports)).saveToEs("spark/docs")
val RDD = sc.esRDD("demo/users")

In [1]:
import org.elasticsearch.spark._
import org.apache.spark.SparkContext    
import org.apache.spark.SparkContext._

In [4]:
val user_df = spark.read.format("es").load("demo/users")
user_df.printSchema()
user_df.select("userId", "name").show(5)

root
 |-- @model: struct (nullable = true)
 |    |-- factor: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- userId: string (nullable = true)

+------+-----------------+
|userId|             name|
+------+-----------------+
|   547|     Betty George|
|   370|   Patrick Browne|
|   587|        John Leal|
|   637|Christopher Weeks|
|   380|   Margaret Moore|
+------+-----------------+
only showing top 5 rows



In [23]:
val movie_df = spark.read.format("es").option("es.read.field.as.array.include", "genres").load("demo/movies")
movie_df.printSchema()
movie_df.select("movieId", "title", "genres", "release_date", "popularity").show(5)

root
 |-- @model: struct (nullable = true)
 |    |-- factor: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- image_url: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- release_date: timestamp (nullable = true)
 |-- title: string (nullable = true)

+-------+--------------------+--------------------+--------------------+----------+
|movieId|               title|              genres|        release_date|popularity|
+-------+--------------------+--------------------+--------------------+----------+
|   4712|      The Wild Child|             [Drama]|1970-02-26 00:00:...|  0.139692|
|   8236|While the City Sl...|  [Drama, Film-Noir]|1956-05-30 00:00:...|  0.

In [12]:
val ratings_df = spark.read.format("es").load("demo/ratings")
ratings_df.printSchema()
ratings_df.show(5)

root
 |-- movieId: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- userId: string (nullable = true)

+-------+------+--------------------+------+
|movieId|rating|           timestamp|userId|
+-------+------+--------------------+------+
|    296|   4.0|1970-01-10 11:23:...|    50|
|    377|   3.0|1970-01-10 11:23:...|    50|
|    454|   3.0|1970-01-10 11:23:...|    50|
|    786|   3.0|1970-01-10 11:23:...|    50|
|   1636|   1.0|1970-01-11 22:45:...|    51|
+-------+------+--------------------+------+
only showing top 5 rows



In [24]:
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.sql.functions.col

In [19]:
val als = new ALS().setUserCol("userId").setItemCol("movieId").setRatingCol("rating").setRegParam(0.1).setRank(20)


In [30]:
val converted_ratings = ratings_df.select(col("userId").cast("int"), col("movieId").cast("int"),col("rating"))
val model = als.fit(converted_ratings)
model.userFactors.show(5)
model.itemFactors.show(5)

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[0.2868985, 0.517...|
| 20|[-0.02282229, -0....|
| 30|[-0.038180783, 0....|
| 40|[0.38619977, 0.06...|
| 50|[0.053539716, -0....|
+---+--------------------+
only showing top 5 rows

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[0.52807945, 0.09...|
| 20|[0.45714593, 0.14...|
| 30|[0.388741, 0.8019...|
| 40|[0.6537328, 0.176...|
| 50|[0.54700166, 0.34...|
+---+--------------------+
only showing top 5 rows



In [33]:
// test out the vector conversion function
val test_vec = model.userFactors.select("features").first().features
print test_vec
print

Name: Unknown Error
Message: <console>:41: error: value features is not a member of org.apache.spark.sql.Row
       val test_vec = model.userFactors.select("features").first().features
                                                                   ^
StackTrace: 

In [31]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions.{ udf, lit }

def convert_vector(x):
    '''Convert a list or numpy array to delimited token filter format'''
    return " ".join(["%s|%s" % (i, v) for i, v in enumerate(x)])
def reverse_convert(s):
    '''Convert a delimited token filter format string back to list format'''
    return  [float(f.split("|")[1]) for f in s.split(" ")]
def vector_to_struct(x, version):
    '''Convert a vector to a SparkSQL Struct with string-format vector and version fields'''
    return (convert_vector(x), version)
vector_struct = udf(vector_to_struct, \
                    StructType([StructField("factor", StringType(), True), \
                                StructField("version", StringType(), True)]))

Name: Unknown Error
Message: <console>:2: error: ':' expected but ')' found.
def convert_vector(x):
                    ^
StackTrace: 